In [72]:
import pandas as pd

print(f"pandas {0}", pd.__version__)

import numpy as np

print(f"numpy {0}", pd.__version__)

import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

from io import StringIO
from support.convertion import load_conversion_from
from support.parsers import p2f
from support.parsers import d2d
from datetime import datetime
from typing import List

plt.rcParams["figure.figsize"] = [14, 10]
plt.style.use("ggplot")

%matplotlib inline
%load_ext autoreload
%autoreload 2

pandas 0 1.1.3
numpy 0 1.1.3
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%reload_ext autoreload

In [130]:
def handle_missing_index(df: pd.DataFrame, indexes: pd.MultiIndex) -> None:
    for idx in indexes:
        print("🔧 Fixing", idx)
        # (Timestamp('2019-09-24 00:00:00+0000', tz='UTC'), 'MLU', 'direct', 'Desktop')
        moment: datetime = idx[0].to_pydatetime()
        site: str = idx[1]
        flow: str = idx[2]
        device: str = idx[3]

        kinds = (
            "linear",
            "index",
            "pad",
            "values",
            "nearest",
            "zero",
            "slinear",
            "quadratic",
            "cubic",
            "barycentric",
            "krogh",
            "akima",
            "from_derivatives",
            "pchip",
            "spline",
            "polynomial",
        )
        
        lower = reference - timedelta(weeks=5)
        upper = reference + timedelta(weeks=4)
        dow: int = df.loc[idx].day_of_week

        result = df[
            (lower < df.index.get_level_values("Fecha"))
            & (upper > df.index.get_level_values("Fecha"))
            & (df.index.get_level_values("flow") == flow)
            & (df.index.get_level_values("Site") == site)
            & (df.index.get_level_values("Device") == device)
            & (df["day_of_week"] == dow)
        ]

        adf: pd.DataFrame = result.interpolate(method="spline", order=2)
        display(df.loc[idx])

In [139]:
df0: pd.DataFrame = pd.read_csv(
    # Cart + Cart
    "datasets/traffic_AR_CL_UY_DC.csv",
    parse_dates=["Fecha"],
    # index_col=["Fecha"],
    usecols=[
        "Fecha",
        "flow",
        "Site",
        "Device",
        "Loading",
        "Shipping",
        "Payments",
        "Review",
        "Congrats",
    ],
    converters={"Device": d2d},
)
df0["Fecha"] = pd.to_datetime(df0["Fecha"], utc=True)

# Some feature engineering
def day_of_week(row) -> int:
    column_name: str = "Fecha"
    return row[column_name].dayofweek if column_name in row else row.name[0].dayofweek

# Some feature engineering
df0["day_of_week"] = df0.apply(day_of_week, axis="columns")  # axis="columns=1|index=0"


index_names: List[str] = ["Fecha", "Site", "flow", "Device"]

sites: List[str] = list(df0["Site"].unique())
flows: List[str] = list(df0["flow"].unique())
devices: List[str] = list(df0["Device"].unique())

criteria = [
    (site, flow, device) for site in sites for flow in flows for device in devices
]

for (site, flow, device) in criteria:
    print(f"⚪ Finding missing dates for  '({site},{flow})' on '{device}' ")

    result: pd.DataFrame = df0.query(
        f""" Site == "{site}" and flow == '{flow}' and Device == '{device}' """
    )
    if len(result.index) < 1:
        print(f"🟢 No value found on '{site}' and flow '{flow}'\n")
        continue

    start: datetime = result["Fecha"].min()
    stop: datetime = result["Fecha"].max()
    print(f"⚪ For '({site},{flow})' on '{device}' time span is [{start} to {stop}]")

    dates: List[datetime] = pd.date_range(
        start=start,  # start on minimun
        end=stop,  # start on maximun
        freq="D",  # frecuency
        tz="UTC",  # timezone
    ).to_list()

    spectrum: pd.MultiIndex = pd.MultiIndex.from_product(
        [dates, [site], [flow], [device]], names=index_names
    )
    result.set_index(index_names, inplace=True)

    missings = spectrum.difference(result.index)
    if len(missings) > 0:
        print(f"🟠 Found:")
        display(missings)
        # TODO: Implement inputter to add this entry to this dataframe
        # handle_missing_index(df0, missings)
    else:
        print(f"🟢 NO missing dates for  '({site},{flow})' on '{device}'\n")

# Setup index on control df1
df1: pd.DataFrame = df0.set_index(index_names)
df1.sort_index(inplace=True)

integer_columns: List[str] = ["Loading", "Shipping", "Payments", "Review", "Congrats"]

for column in [cl for cl in df1.columns if cl in integer_columns]:
    df1[column] = df1[column].fillna(0).astype(np.int64)

⚪ Finding missing dates for  '(MLA,cart)' on 'Android' 
⚪ For '(MLA,cart)' on 'Android' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,cart)' on 'Android'

⚪ Finding missing dates for  '(MLA,cart)' on 'Desktop' 
⚪ For '(MLA,cart)' on 'Desktop' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,cart)' on 'Desktop'

⚪ Finding missing dates for  '(MLA,cart)' on 'Mobile' 
⚪ For '(MLA,cart)' on 'Mobile' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟠 Found:


MultiIndex([('2019-08-15 00:00:00+00:00', 'MLA', 'cart', 'Mobile')],
           names=['Fecha', 'Site', 'flow', 'Device'])

⚪ Finding missing dates for  '(MLA,cart)' on 'iOS' 
⚪ For '(MLA,cart)' on 'iOS' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,cart)' on 'iOS'

⚪ Finding missing dates for  '(MLA,direct)' on 'Android' 
⚪ For '(MLA,direct)' on 'Android' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,direct)' on 'Android'

⚪ Finding missing dates for  '(MLA,direct)' on 'Desktop' 
⚪ For '(MLA,direct)' on 'Desktop' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,direct)' on 'Desktop'

⚪ Finding missing dates for  '(MLA,direct)' on 'Mobile' 
⚪ For '(MLA,direct)' on 'Mobile' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟠 Found:


MultiIndex([('2019-08-15 00:00:00+00:00', 'MLA', 'direct', 'Mobile')],
           names=['Fecha', 'Site', 'flow', 'Device'])

⚪ Finding missing dates for  '(MLA,direct)' on 'iOS' 
⚪ For '(MLA,direct)' on 'iOS' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLA,direct)' on 'iOS'

⚪ Finding missing dates for  '(MLC,cart)' on 'Android' 
🟢 No value found on 'MLC' and flow 'cart'

⚪ Finding missing dates for  '(MLC,cart)' on 'Desktop' 
⚪ For '(MLC,cart)' on 'Desktop' time span is [2020-06-25 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLC,cart)' on 'Desktop'

⚪ Finding missing dates for  '(MLC,cart)' on 'Mobile' 
⚪ For '(MLC,cart)' on 'Mobile' time span is [2020-06-25 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLC,cart)' on 'Mobile'

⚪ Finding missing dates for  '(MLC,cart)' on 'iOS' 
🟢 No value found on 'MLC' and flow 'cart'

⚪ Finding missing dates for  '(MLC,direct)' on 'Android' 
⚪ For '(MLC,direct)' on 'Android' time span is [2019-03-28 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(

MultiIndex([('2019-09-24 00:00:00+00:00', 'MLU', 'direct', 'Desktop')],
           names=['Fecha', 'Site', 'flow', 'Device'])

⚪ Finding missing dates for  '(MLU,direct)' on 'Mobile' 
⚪ For '(MLU,direct)' on 'Mobile' time span is [2019-03-01 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟠 Found:


MultiIndex([('2019-09-24 00:00:00+00:00', 'MLU', 'direct', 'Mobile')],
           names=['Fecha', 'Site', 'flow', 'Device'])

⚪ Finding missing dates for  '(MLU,direct)' on 'iOS' 
⚪ For '(MLU,direct)' on 'iOS' time span is [2019-03-28 00:00:00+00:00 to 2020-06-30 00:00:00+00:00]
🟢 NO missing dates for  '(MLU,direct)' on 'iOS'



In [43]:
def day_of_week(row) -> int:
    return row.name[0].dayofweek


# Some feature engineering
df1["day_of_week"] = df1.apply(day_of_week, axis="columns")  # axis="columns=1|index=0"


def month_of(row) -> int:
    return row.name[0].to_pydatetime().month


df1["month"] = df1.apply(month_of, axis="columns")


def year_of(row) -> int:
    dt: datetime = row.name[0].to_pydatetime()
    return dt.year


df1["year"] = df1.apply(year_of, axis="columns")


df1["input_on_shipping"] = df1["Shipping"] > df1["Loading"]
df1["missing_on_shipping"] = df1["Shipping"] < 1

df1["input_on_payments"] = df1["Payments"] > df1["Shipping"]
df1["missing_on_payments"] = df1["Payments"] < 1

df1["input_on_review"] = df1["Review"] > df1["Payments"]
df1["missing_on_review"] = df1["Review"] < 1

df1["input_on_congrats"] = df1["Congrats"] > df1["Review"]
df1["missing_on_congrats"] = df1["Congrats"] < 1

df1

Loading  Shipping  Payments  \
Fecha                     Site flow   Device                                 
2019-03-01 00:00:00+00:00 MLA  cart   Android        0      7084      5313   
                                      Desktop        0      5419      4177   
                                      Mobile         0      2426      1832   
                                      iOS            0      1179       856   
                               direct Android   160081    152310    124551   
...                                                ...       ...       ...   
2020-06-30 00:00:00+00:00 MLC  direct iOS        29876     29952     25386   
                          MLU  direct Android    30437     30534     24870   
                                      Desktop     9859      9767      8177   
                                      Mobile      8416      8267      6636   
                                      iOS         9338      9372      8238   

                                               Review  Congrats  day_of_week  \
Fecha                     Site flow   Device                                   
2019-03-01 00:00:00+00:00 MLA  cart   Android    3981      3731            4   
                                      Desktop    3580      3479            4   
                                      Mobile     1373      1290            4   
                                      iOS         671       622            4   
                               direct Android   89380     83856            4   
...                                               ...       ...          ...   
2020-06-30 00:00:00+00:00 MLC  direct iOS       21049     19642            1   
                          MLU  direct Android   18660     17698            1   
                                      Desktop    6690      6566            1   
                                      Mobile     4899      4626            1   
                                      iOS        6333      5959            1   

                                               month  year  input_on_shipping  \
Fecha                     Site flow   Device                                    
2019-03-01 00:00:00+00:00 MLA  cart   Android      3  2019               True   
                                      Desktop      3  2019               True   
                                      Mobile       3  2019               True   
                                      iOS          3  2019               True   
                               direct Android      3  2019              False   
...                                              ...   ...                ...   
2020-06-30 00:00:00+00:00 MLC  direct iOS          6  2020               True   
                          MLU  direct Android      6  2020               True   
                                      Desktop      6  2020              False   
                                      Mobile       6  2020              False   
                                      iOS          6  2020               True   

                                               missing_on_shipping  \
Fecha                     Site flow   Device                         
2019-03-01 00:00:00+00:00 MLA  cart   Android                False   
                                      Desktop                False   
                                      Mobile                 False   
                                      iOS                    False   
                               direct Android                False   
...                                                            ...   
2020-06-30 00:00:00+00:00 MLC  direct iOS                    False   
                          MLU  direct Android                False   
                                      Desktop                False   
                                      Mobile                 False   
                                      iOS                    False   

                          

In [42]:
df1[ df1["missing_on_review"] ]

Loading  Shipping  Payments  \
Fecha                     Site flow   Device                                 
2019-03-22 00:00:00+00:00 MLA  cart   Android        0      7990      8907   
                                      iOS            0      1436      1455   
                               direct Android   179146    169846    138177   
                                      iOS        32520     30281     25882   
2019-08-27 00:00:00+00:00 MLA  cart   Android        0     10936      7786   
                                      iOS            0      1870      1398   
                               direct Android   236701    223993    180928   
                                      iOS        42192     39303     39753   
                          MLC  direct Android    38736     38688     39303   
                                      iOS         8931      8601      6846   
                          MLU  direct Android    16296     16348     17051   
                                      iOS         4972      4017      4158   
2019-12-05 00:00:00+00:00 MLA  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   
                          MLC  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   
                          MLU  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   
2019-12-06 00:00:00+00:00 MLA  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   
                          MLC  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   
                          MLU  direct Android        0         0         0   
                                      Desktop        0         0         0   
                                      Mobile         0         0         0   
                                      iOS            0         0         0   

                                               Review  Congrats  day_of_week  \
Fecha                     Site flow   Device                                   
2019-03-22 00:00:00+00:00 MLA  cart   Android       0      4044            4   
                                      iOS           0       763            4   
                               direct Android       0     89890            4   
                                      iOS           0     18718            4   
2019-08-27 00:00:00+00:00 MLA  cart   Android       0      5050            1   
                                      iOS           0      1011            1   
                               direct Android       0    113464            1   
                                      iOS           0     22955            1   
                          MLC  direct Android       0     17780            1   
                                      iOS           0      4804            1   
                          MLU  direct Android       0      9628            1   
                                      iOS 